# 00. CAN SLIM 한국 주식시장 스크리닝 튜토리얼

## 소개
이 튜토리얼 시리즈는 윌리엄 오닐의 **CAN SLIM** 투자 전략을 한국 주식시장에 적용하는 방법을 단계별로 안내합니다.

### CAN SLIM이란?
CAN SLIM은 성장주 투자의 대가 윌리엄 오닐(William J. O'Neil)이 개발한 주식 선별 전략입니다.

각 글자가 의미하는 바:
- **C** (Current Earnings): 현재 분기 EPS 성장률
- **A** (Annual Earnings): 연간 EPS 성장률
- **N** (New): 신제품, 신경영진, 신고가
- **S** (Supply and Demand): 수급 상황
- **L** (Leader or Laggard): 업종 내 선도주 여부
- **I** (Institutional Sponsorship): 기관투자자 매수
- **M** (Market Direction): 전체 시장 방향

### 한국 시장 적용 시 고려사항
1. **데이터 소스**: pykrx를 통한 한국거래소 데이터 활용
2. **지표 변환**: 미국 시장 기준을 한국 시장에 맞게 조정
3. **추가 지표**: 한국 시장 특성 반영 (외국인 매수 동향 등)

## 환경 설정

### 필수 라이브러리 설치
아래 셀을 실행하여 필요한 라이브러리를 설치합니다.

In [ ]:
# 필수 라이브러리 설치
!pip install pykrx pandas numpy matplotlib seaborn plotly
!pip install ta  # 기술적 지표 계산용
!pip install tqdm  # 진행률 표시용
!pip install openpyxl  # Excel 파일 처리용

### 라이브러리 임포트 및 설정

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 데이터 수집
from pykrx import stock

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 기술적 지표
import ta

# 유틸리티
from tqdm import tqdm
import time

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# Pandas 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("라이브러리 로드 완료!")
print(f"pykrx 버전: {stock.__version__ if hasattr(stock, '__version__') else 'Unknown'}")

## 기본 함수 정의

### 날짜 관련 유틸리티

In [ ]:
def get_date_strings(months_ago=12):
    """
    현재 날짜와 과거 날짜를 문자열로 반환
    
    Parameters:
    months_ago (int): 몇 개월 전 날짜를 가져올지
    
    Returns:
    tuple: (시작일, 종료일) 형식의 문자열 튜플
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=months_ago*30)
    
    return start_date.strftime('%Y%m%d'), end_date.strftime('%Y%m%d')

def get_quarter_dates():
    """
    최근 5개 분기의 시작일과 종료일 반환
    """
    today = datetime.now()
    quarters = []
    
    for i in range(5):
        quarter_end = today - timedelta(days=90*i)
        quarter_start = quarter_end - timedelta(days=90)
        quarters.append({
            'start': quarter_start.strftime('%Y%m%d'),
            'end': quarter_end.strftime('%Y%m%d'),
            'label': f'Q{4-i}'
        })
    
    return quarters[::-1]  # 오래된 순서로 정렬

# 테스트
start, end = get_date_strings(12)
print(f"데이터 수집 기간: {start} ~ {end}")
print("\n분기별 날짜:")
for q in get_quarter_dates():
    print(f"{q['label']}: {q['start']} ~ {q['end']}")

### 데이터 수집 기본 함수

In [ ]:
def get_stock_list(market="KOSPI"):
    """
    주식 종목 리스트 가져오기
    
    Parameters:
    market (str): 'KOSPI' 또는 'KOSDAQ'
    
    Returns:
    DataFrame: 종목 코드와 이름
    """
    tickers = stock.get_market_ticker_list(market=market)
    
    stock_list = []
    for ticker in tqdm(tickers[:10], desc=f"{market} 종목 정보 수집"):  # 데모용으로 10개만
        try:
            name = stock.get_market_ticker_name(ticker)
            stock_list.append({
                'ticker': ticker,
                'name': name,
                'market': market
            })
            time.sleep(0.1)  # API 호출 제한 방지
        except:
            continue
    
    return pd.DataFrame(stock_list)

# 테스트: KOSPI 상위 10개 종목
kospi_stocks = get_stock_list("KOSPI")
print(f"\n수집된 KOSPI 종목 수: {len(kospi_stocks)}")
print(kospi_stocks.head())

## CAN SLIM 스크리닝 기준 정리

### 핵심 정량적 기준

In [ ]:
# CAN SLIM 기준 설정
CANSLIM_CRITERIA = {
    'C': {
        'quarterly_eps_growth': 0.25,  # 분기 EPS 25% 이상 성장
        'quarterly_sales_growth': 0.25,  # 분기 매출 25% 이상 성장
        'description': '현재 분기 실적 (Current Quarterly Earnings)'
    },
    'A': {
        'annual_eps_growth': 0.25,  # 연간 EPS 25% 이상 성장
        'eps_stability': 3,  # 최근 3년 연속 EPS 증가
        'roe_threshold': 0.17,  # ROE 17% 이상
        'description': '연간 실적 (Annual Earnings)'
    },
    'N': {
        'price_near_high': 0.85,  # 52주 최고가 대비 85% 이상
        'new_high_volume': 1.5,  # 신고가 시 거래량 1.5배 이상
        'description': '신고가, 신제품, 신경영진 (New)'
    },
    'S': {
        'shares_outstanding': 50000000,  # 유통주식수 5천만주 이하 선호
        'volume_increase': 1.5,  # 평균 거래량 대비 1.5배 증가
        'description': '수급 (Supply and Demand)'
    },
    'L': {
        'relative_strength': 80,  # 상대강도 80 이상 (상위 20%)
        'industry_rank': 0.2,  # 업종 내 상위 20%
        'description': '업종 선도주 (Leader)'
    },
    'I': {
        'institutional_ownership': 0.1,  # 기관 보유 10% 이상
        'institutional_increase': True,  # 기관 매수 증가 추세
        'description': '기관 투자자 (Institutional Sponsorship)'
    },
    'M': {
        'market_trend': 'uptrend',  # 시장 상승 추세
        'index_confirmation': True,  # 주요 지수 상승 확인
        'description': '시장 방향성 (Market Direction)'
    }
}

# 기준 출력
print("CAN SLIM 스크리닝 기준")
print("="*50)
for key, criteria in CANSLIM_CRITERIA.items():
    print(f"\n{key} - {criteria['description']}")
    for k, v in criteria.items():
        if k != 'description':
            print(f"  • {k}: {v}")

## 튜토리얼 구성 안내

### 노트북 구성

In [ ]:
tutorials = [
    {
        'notebook': '01_data_collection.ipynb',
        'title': '데이터 수집',
        'description': 'pykrx를 활용한 주가, 재무, 투자자별 데이터 수집 방법',
        'key_topics': ['주가 데이터', '재무 데이터', '투자자별 매매 동향']
    },
    {
        'notebook': '02_current_earnings_analysis.ipynb',
        'title': 'C - 현재 분기 실적 분석',
        'description': '분기별 EPS, 매출 성장률 계산 및 가속화 패턴 분석',
        'key_topics': ['분기 EPS 성장률', '매출 성장률', '실적 가속화']
    },
    {
        'notebook': '03_annual_earnings_analysis.ipynb',
        'title': 'A - 연간 실적 분석',
        'description': '연간 EPS 성장률, CAGR, ROE 분석',
        'key_topics': ['연간 성장률', 'CAGR 계산', 'ROE 분석']
    },
    {
        'notebook': '04_new_highs_technical.ipynb',
        'title': 'N - 신고가 및 기술적 분석',
        'description': '52주 신고가 근접성, 차트 패턴, 거래량 분석',
        'key_topics': ['52주 고가', '차트 패턴', '거래량 급증']
    },
    {
        'notebook': '05_supply_demand_analysis.ipynb',
        'title': 'S - 수급 분석',
        'description': '유통주식수, 거래량 변화, 수급 상황 분석',
        'key_topics': ['유통주식수', '거래량 분석', '수급 지표']
    },
    {
        'notebook': '06_leader_laggard_analysis.ipynb',
        'title': 'L - 업종 선도주 분석',
        'description': '상대강도(RS), 업종 내 순위, 모멘텀 분석',
        'key_topics': ['상대강도', '업종 순위', '모멘텀']
    },
    {
        'notebook': '07_institutional_sponsorship.ipynb',
        'title': 'I - 기관 투자자 분석',
        'description': '기관 및 외국인 매매 동향, 보유 비중 분석',
        'key_topics': ['기관 매수', '외국인 동향', '스마트머니']
    },
    {
        'notebook': '08_market_direction.ipynb',
        'title': 'M - 시장 방향성 분석',
        'description': 'KOSPI/KOSDAQ 추세, 시장 타이밍 분석',
        'key_topics': ['지수 분석', '시장 추세', '타이밍']
    },
    {
        'notebook': '09_complete_screening.ipynb',
        'title': '통합 스크리닝 시스템',
        'description': '모든 지표를 통합한 자동 스크리닝 시스템 구축',
        'key_topics': ['통합 스크리닝', '점수 산출', '포트폴리오']
    }
]

# 튜토리얼 구성 표시
tutorials_df = pd.DataFrame(tutorials)
print("\n📚 CAN SLIM 튜토리얼 구성")
print("="*80)
for idx, row in tutorials_df.iterrows():
    print(f"\n{idx+1}. {row['title']}")
    print(f"   📝 파일: {row['notebook']}")
    print(f"   📖 설명: {row['description']}")
    print(f"   🔑 주요 내용: {', '.join(row['key_topics'])}")

## 샘플 데이터로 간단한 예제

### 삼성전자 데이터로 기본 분석 예시

In [ ]:
# 삼성전자 데이터 가져오기
ticker = "005930"  # 삼성전자
start_date, end_date = get_date_strings(3)  # 최근 3개월

print(f"삼성전자({ticker}) 데이터 수집 중...")

# 주가 데이터
price_data = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
print(f"\n주가 데이터 수집 완료: {len(price_data)}일")
print(price_data.tail())

# 기본 재무 데이터
fundamental = stock.get_market_fundamental_by_date(start_date, end_date, ticker)
print(f"\n재무 데이터 수집 완료")
print(fundamental.tail())

### 간단한 시각화

In [ ]:
# 주가 차트 그리기
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.7, 0.3],
    subplot_titles=('주가', '거래량')
)

# 캔들스틱 차트
fig.add_trace(
    go.Candlestick(
        x=price_data.index,
        open=price_data['시가'],
        high=price_data['고가'],
        low=price_data['저가'],
        close=price_data['종가'],
        name='주가'
    ),
    row=1, col=1
)

# 거래량
fig.add_trace(
    go.Bar(
        x=price_data.index,
        y=price_data['거래량'],
        name='거래량',
        marker_color='lightblue'
    ),
    row=2, col=1
)

fig.update_layout(
    title=f'삼성전자 주가 차트 ({start_date} ~ {end_date})',
    xaxis2_title='날짜',
    yaxis_title='주가 (원)',
    yaxis2_title='거래량',
    height=600,
    showlegend=False
)

fig.update_xaxes(rangeslider_visible=False)
fig.show()

## 마무리

이 노트북에서는 CAN SLIM 전략의 기본 개념과 환경 설정을 완료했습니다.

### 다음 단계
1. **01_data_collection.ipynb**: pykrx를 활용한 체계적인 데이터 수집 방법 학습
2. 각 CAN SLIM 지표별 상세 분석 방법 학습
3. 통합 스크리닝 시스템 구축

### 참고사항
- API 호출 제한을 피하기 위해 적절한 딜레이를 추가하세요
- 대량 데이터 수집 시 데이터를 로컬에 저장하여 재사용하세요
- 실제 투자 결정은 추가적인 검증과 리스크 관리가 필요합니다

**Happy Investing! 🚀**